# Animations in Makie

Experimenting with `Makie` [animations](https://docs.makie.org/v0.18.0/documentation/animation/index.html) to see if it's viable.

In [1]:
using GLMakie

# Use a custom theme for fun
cyberpunk_theme = Theme(
        # teal/cyan, pink, yellow, matrix green, red, violet
        palette = (color =["#08F7FE", "#FE53BB", "#F5D300", "#00ff41", :red, "#9467bd"],),
        backgroundcolor = "#212946",
        textcolor=:gray90,
        resolution=(640,320),
        fontsize=12,
        font = "Arial",
        colormap = :cool,
        cycle = Cycle([:color]),
        Axis = ( 
            backgroundcolor = "#212946",
            topspinevisible = false,
            rightspinevisible = false,
            xgridcolor = "#2A3459",
            ygridcolor = "#2A3459",
            bottomspinecolor = "#2A3459",
            leftspinecolor = "#2A3459",
            ytickcolor = "#2A3459",
            xtickcolor = "#2A3459",
            titlesize=16,
            yautolimitmargin=(0.05, 0.1), # Leaves room for label text
            ),
        Legend = (
            bgcolor = "#212946",
            framecolor = "#2A3459",
        ),            
        # Note in order to apply specific settings to plot types we need to use the Struct/type
        # so to theme barplots() you need to use the BarPlot type
        Lines = (
            linewidth=2,),
        BarPlot = (
            cycle=Cycle([:color]), # Interesting that I have to put it here
            label_size=12,
        )
)

set_theme!(cyberpunk_theme)

From the docs:

> To create an animation you need to use the `record` function.

> * First you create a `Figure`
> * Next, you pass a function that modifies this figure frame-by-frame to record. Any changes you make to the figure or its plots will appear in the final animation. 
> * You also need to pass an iterable which has as many elements as you want frames in your animation. The function that you pass as the first argument is called with each element from this iterator over the course of the animation.

## A simple goal: Create a dot that moves up and to the right

Let's try to keep this example really, really simple: animate a dot that's moving up and to the right as time passes by. 

To do that, I think we have to understand two key concepts: `Observable`s and listeners:

### Observables

An `Observable` is something that `Makie` is designed to 'listen' to and react to changes. We declare an `Observable` like below:

In [50]:
time = Observable(0.0)

Observable{Float64} with 0 listeners. Value:
0.0

The above code created an `Observable` called `time` that currently has the value of 0.0. We can update the current value of `time` by simply using this *empty* bracket notation:

In [51]:
time[] = 10
time

Observable{Float64} with 0 listeners. Value:
10.0

### Create listeners:

Use the `@lift` macro to define a relationship with an `Observable`. 

In this example, both the `x` and `y` coordinates will depend only on `time`:

In [52]:
x = @lift(1 + $time)
y = @lift(5 + 0.5*($time))
# look at time
time

Observable{Float64} with 2 listeners. Value:
10.0

If we look at `time` again, we should see that it has 2 listeners now.

Then we need to follow the 3 steps to create our animation:

* create figure: `fig`
* iterator: `timestamps`
* function: `move_dot`
    * Changes the value of `time`, which in turn, changes `x` and `y`

In [39]:
# 1. Create a figure
fig = scatter(x, y, color = :red, markersize = 20,
    axis = (title = @lift("t = $(round($time, digits =1))"),))

# Next, pass a function that modifies this figure frame by frame to record.
"""Set the observable `time` equal to t"""
function move_dot(t)
    time[] = t
end

# pass an iterable that has the same number of elements as frames in the animation
framerate = 30
timestamps = range(0, 5, step=1/30)

# Create an animation
record(move_dot, fig, "moving_dot.gif", timestamps;
        framerate = framerate) 

# # Alternate with `do` notation:
# record(fig, "moving_dot.gif", timestamps;
#         framerate = framerate) do t
#     time[] = t
# end

"moving_dot.gif"

What this code does:
* for every `t` in the iterator `timestamps` which is a range between 0 and 5
* call the `move_dot` function with a single `t` as input
* each frame of animation will be based on each visualization after `move_dot` changes the `x` and `y` values

![img](moving_dot.gif)

Instead of setting `x` and `y` as listeners that depend on `time`, we could instead change the type of `Observable` so that it's a 2d point.

What about using a `Point2f` type as the `Observable`?

In [53]:
location = Observable(Point2f(0,0))

# 1. Create a figure
fig, ax, scatterplot = scatter(location, color = :red, markersize = 20,
    axis = (title = @lift("t = $($location)"),))

# Pro tip: The scale of the axis doesn't update automatically as the dot moves,
# so manually set your limits!
limits!(ax, 0, 30, 0, 30)

# Next, pass a function that modifies this figure frame by frame to record.
"""Set the observable `time` equal to t"""
function move_dot2(i)
    location[] = location[] + [0.5, 0.75]
end

# pass an iterable that has the same number of elements as frames in the animation
framerate = 30
frames = 1:30

# Create an animation
record(move_dot2, fig, "moving_dot2.gif", frames;
        framerate = framerate) 

"moving_dot2.gif"

![img](moving_dot2.gif)